<h1>Mall customers dataset </h1>

<h4> The aim of this exercise is to segment the customers based on their age, gender, and interest. This is called "Customer segmentation" and is an important practise of dividing customers based on individual groups that are similar. It is useful in customised marketing.</h4>

<h4>Customer segmentation</h4>
<p>Customer Segmentation is the process of division of customer base into several groups of individuals that share a similarity in different ways that are relevant to marketing such as gender, age, interests, and miscellaneous spending habits.

Companies that deploy customer segmentation are under the notion that every customer has different requirements and require a specific marketing effort to address them appropriately. Companies aim to gain a deeper approach of the customer they are targeting. Therefore, their aim has to be specific and should be tailored to address the requirements of each and every individual customer. Furthermore, through the data collected, companies can gain a deeper understanding of customer preferences as well as the requirements for discovering valuable segments that would reap them maximum profit. This way, they can strategize their marketing techniques more efficiently and minimize the possibility of risk to their investment.

The technique of customer segmentation is dependent on several key differentiators that divide customers into groups to be targeted. Data related to demographics, geography, economic status as well as behavioral patterns play a crucial role in determining the company direction towards addressing the various segments.</p>

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans 
from sklearn.metrics import silhouette_score
import seaborn as sns

In [ ]:
cust = pd.read_csv('/kaggle/input/mall-customers/Mall_Customers.csv')
cust.head()

## Let's fist do basic visualisation to understand the data and the distribution of the different variables

#### Gender

In [ ]:
#let's look at the men and women via histogram

sns.countplot(x='Genre', data=cust)
plt.title('Customer Gender')

In [ ]:
#to make a piechart:
gender=cust.Genre.value_counts()
gender_label=['Female','Male']
plt.pie(gender, labels=gender_label, autopct='%0.2f%%',startangle=90)
plt.title('Distribution of men and women within the customers of the JCPenney')
plt.show()

#### Age distribution

In [ ]:
#let's see the max and min of ages
cust.describe()

<p>The minimum age is 18 and the maximum is 70. We can create 6 bins to group people by age group. Each bin could represent 10 years</p>

In [ ]:
bin_list=[10,20,30,40,50,60,70]
plt.hist(cust['Age'], bins=bin_list, rwidth=0.9)
plt.xlabel('Age')
plt.ylabel('frequency')
plt.title('Age distribution of customers')

#### Annual income of customers

In [ ]:
plt.hist(cust['Annual Income (k$)'], bins=12, rwidth=0.9)
plt.xlabel("Income in 1000's of $")
plt.ylabel("frequency")
plt.title('Annual income of customers')

#### Spending score of the customers

In [ ]:
plt.hist(cust['Spending Score (1-100)'], bins=[0,10,20,30,40,50,60,70,80,90,100], rwidth=0.9)
plt.xlabel("Spending score")
plt.ylabel("frequency")
plt.title('Spending Score of customers')

<p>The gender in this dataset is a categorical variable. k-means algorithm isn't directly applicable to categorical variables because Euclidean distance function isn't really meaningful for discrete variables. So, lets change these variables and transform them to an indicator variable (or dummy variable). This is a numerical variable used to label categories. They are called 'dummies' because the numbers themselves don't have inherent meaning. In our case, let's change male to "0" and female to "1".</p>

In [ ]:
#let's also drop the customer ID because it's not important
cust.drop("CustomerID", axis = 1, inplace=True)
#cust.drop("Genre", axis = 1, inplace=True)


cust["Genre"].replace("Male", 0, inplace=True)
cust["Genre"].replace("Female", 1, inplace=True)
cust

## Modelling

### Let's start by visualising the relationship between different variable groups.
<p>We are interested primarily in those who have a high spending score because this is the category we want to keep as customers for the mall. So let's check if there is a relationship between age and spending score, and annual income and spending score.</p>

### Gender and spending

In [ ]:
plt.scatter(cust['Genre'], cust['Spending Score (1-100)'])

<p>It's hard to see clusters or relationships in this graph. That's mainly because the gender category has two distinct variables. Therefore, we will remove the gender from our analysis in order to make it simpler.</p>

### Age and spending

In [ ]:
plt.scatter(cust['Age'], cust['Spending Score (1-100)'])

<p>It appears that there is some sort of correlation between being younger (less than 35 yo) and spending more, while the relatively older people are spending less than 60%. This graphs shows therefore 2 clusters.
</p>

### Income and spending

In [ ]:
plt.scatter(cust['Annual Income (k$)'], cust['Spending Score (1-100)'])

<p>In this graph, however, it's obvious that there are 5 groups, or clusters, in this dataset if we compare annual income and the spending score, and this is probably what we are interested in seing. Let's evaluate this more below.</p>

## K-mean clustering

<p>Lets apply k-means on our dataset, and take look at cluster labels.</p>

The KMeans class has many parameters that can be used, but we will be using these three:

<ul>
    <li> <b>init</b>: Initialization method of the centroids. </li>
    <ul>
        <li> Value will be: "k-means++" </li>
        <li> k-means++: Selects initial cluster centers for k-mean clustering in a smart way to speed up convergence.</li>
    </ul>
    <li> <b>n_clusters</b>: The number of clusters to form as well as the number of centroids to generate. </li>
    <ul> <li> Value will be: 5 (we can change this anytime if we want to test more values)</li> </ul>
    <li> <b>n_init</b>: Number of time the k-means algorithm will be run with different centroid seeds. The final results will be the best output of n_init consecutive runs in terms of inertia. </li>
    <ul> <li> Value will be: automatic but can be chosen if needed </li> </ul>
</ul>

Let's initialize KMeans with these parameters, where the output parameter is called <b>k_means</b>.

### Age vs spending clustering
<p>Let's test cluster number 2 to verify what we saw in the 'Age' vs 'Spending score' graph.
First, we will only keep the Age and the Spending score column to simplify the dimensions and keep them in 2D.
</p>

In [ ]:
#Let's have a new dataframe first with only the Age and the spending score

cust_age=cust.drop(["Annual Income (k$)", "Genre"], axis = 1)

In [ ]:
#we can test a cluster number 2 to verify what we saw in the 'Age' vs 'Spending score' graph. 
#However, we will use 4 clusters here as we saw in the elbow plot that 4 is the optimal number. See below

k_means_age=KMeans(n_clusters=4)

#We can also use this code below in case we want to determine the n_init number
#k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 20)

k_means_age.fit(cust_age)
labels = k_means_age.labels_
print(labels)

<p>Let's see where is the location of the centers

In [ ]:
centers_age=k_means_age.cluster_centers_
centers_age

<p>Let's plot a graph to visualise this relationship 

In [ ]:
plt.figure(figsize=(10, 8))

plt.scatter(cust_age['Age'], 
            cust_age['Spending Score (1-100)'], 
            c=k_means_age.labels_, s=100)

plt.scatter(centers_age[:,0], centers_age[:,1], color='blue', marker='s', s=200) 

plt.xlabel('Age')
plt.ylabel('Spending Score')
plt.title('K-Means with 2 clusters')

plt.show()

Let's measure the silhouette score of this clustering:

The silhouette value is a measure of how similar an object is to its own cluster (cohesion) compared to other clusters (separation). The silhouette ranges from −1 to +1, where a high value indicates that the object is well matched to its own cluster and poorly matched to neighboring clusters. If most objects have a high value, then the clustering configuration is appropriate. If many points have a low or negative value, then the clustering configuration may have too many or too few clusters.

In [ ]:
score = silhouette_score (cust_age, k_means_age.labels_)

print("The score is = ", score)

<p>This graph shows that we can have 4 clusters based on the Age and the spending score. Therefore, we can have 4 groups:
    <ul>
        <li>Younger people with high spending score</li>
        <li>younger people with average spending score</li>
        <li>People with low spending score of less that 60 that belong to all age groups</li>
        <li>Older people with average spending score.</li>
    </ul>
The score of 0.5 is good. Let's see if we can get it higher.

Let's what is the optimal number of clusters by constructing an elbow plot

In [ ]:
elbowlist1 = []
for i in range(1,15): 
    k_means_age = KMeans(n_clusters=i, init="k-means++",random_state=0)
    k_means_age.fit(cust_age)
    elbowlist1.append(k_means_age.inertia_)  

plt.plot(range(1,15),elbowlist1,marker="*",c="black")
plt.title("Elbow plot for optimal number of clusters: age and spending")

Here we can make sure that the cluster number 4 that we chose is correct. The elbow plot has a distinct slope break at 4 indicating that 4 is the optimal cluster number when comparing between age and spending. It's good to construct this plot to make sure that the analysis is ok. We can rerun our analysis by updating the number of clusters based on this plot.

### Annual income vs spending clustering
<p>Let's test now cluster number 5 to verify what we saw in the 'Annual income' vs 'Spending score' graph.
First, we will only keep the Annual income and the Spending score columns to simplify the dimensions and keep them in 2D.
</p>

In [ ]:
#we drop the annual income column
cust_income=cust.drop(["Age", "Genre"], axis = 1)

In [ ]:
#let's test cluster number 2 to verify what we saw in the 'Age' vs 'Spending score' graph.

k_means_income=KMeans(n_clusters=5)

#We can also use this code below in case we want to determine the n_init number
#k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 20)

k_means_income.fit(cust_income)
labels = k_means_income.labels_
print(labels)

<p>Let's see where is the location of the centers

In [ ]:
centers_income=k_means_income.cluster_centers_
centers_income

<p>Let's plot a graph to visualise this relationship 

In [ ]:
plt.figure(figsize=(10, 8))

plt.scatter(cust_income['Annual Income (k$)'], 
            cust_income['Spending Score (1-100)'], 
            c=k_means_income.labels_, s=100)

plt.scatter(centers_income[:,0], centers_income[:,1], color='blue', marker='s', s=200) 

plt.xlabel('Annual Income in K$')
plt.ylabel('Spending Score')
plt.title('K-Means with 5 clusters')

plt.show()

In [ ]:
score_2 = silhouette_score (cust_income, k_means_income.labels_)

print("The score is = ", score_2)

<p>Here we can see in a much clearer way that there are 5 different clusters belonging to five different groups:
    <ul>
        <li>Low annual income and high spending score --> Interesting category</li>
        <li>High annual income and high spending score --> interesting category</li>
        <li>Low annual income and low spending score --> Not interesting at all </li>
        <li>High annual income and low spending score </li>
        <li>Middge annual income and middle spending score </li>
    </ul>
    
Here we have silhouette score of 0.55 which is better than before. That means that this clustering fits better than the age vs spending one calculated above.
</p>

Let's just make sure that 5 is a good number of clusters by constructing an elbow plot

In [ ]:
elbowlist2 = []
for i in range(1,15): 
    k_means_income = KMeans(n_clusters=i, init="k-means++",random_state=0)
    k_means_income.fit(cust_income)
    elbowlist2.append(k_means_income.inertia_)  

plt.plot(range(1,15),elbowlist2,marker="*",c="black")
plt.title("Elbow plot for optimal number of clusters: income and spending")

Here we can make sure that the cluster number 5 that we chose is correct. The elbow plot has a distinct slope break at 5 indicating that 5 is the optimal cluster number when comparing between income and spending.

#### Visualise the gender distribution in this clustering
<p>However, we might be interested to see if gender plays a role in this classification. We will therefore color some blobs by gender.

In [ ]:
ax=plt.figure(figsize=(10, 8))

scatter=plt.scatter(cust_income['Annual Income (k$)'], 
            cust_income['Spending Score (1-100)'], 
            c=cust['Genre'], s=100)

plt.scatter(centers_income[:,0], centers_income[:,1], color='blue', marker='s', s=200) 

legend1 = ax.legend(*scatter.legend_elements(), loc="right", title="Gender")
ax.add_artist(legend1)

plt.xlabel('Annual Income in K$')
plt.ylabel('Spending Score')
plt.title('K-Means with 5 clusters')

plt.show()

<P>There isn't much difference between gender. One can argue that females might have a higher spending score than men because there are relatively more yellow than purple in the high spending score categories. However, note that there are more females in this dataset than men (56% against 44%) so it's normal to have more females in this graph. Therefore, the gender does not have a noticeable role in this classification.
    
Remember:
<ul>
    <li>male = 0 </li>
    <li>Female = 1 </li>

### Evaluating both age and annual income on clustering

<p>We will use three dimensions for this analysis and visualise results in a 3D graph. We will have an interactive 3D graph to help with visualisation.

In [ ]:
#we drop the gender column

cust_3D=cust.drop("Genre", axis = 1)

In [ ]:
k_means_3D=KMeans(n_clusters=5)

#We can also use this code below in case we want to determine the n_init number
#k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 20)

k_means_3D.fit(cust_3D)
labels = k_means_3D.labels_
print(labels)

In [ ]:
centers_3D=k_means_3D.cluster_centers_
centers_3D

In [ ]:
from matplotlib import interactive
interactive(True)

%matplotlib qt

from mpl_toolkits.mplot3d import Axes3D 
fig = plt.figure(1, figsize=(8, 6))
plt.clf()
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)

plt.cla()
#plt.ylabel('Age', fontsize=18)
#plt.xlabel('Income', fontsize=16)
#plt.zlabel('Education', fontsize=16)
ax.set_xlabel('Age')
ax.set_ylabel('Income in 1000s $')
ax.set_zlabel('Spending')

ax.scatter(cust_3D['Age'], cust_3D['Annual Income (k$)'], cust_3D['Spending Score (1-100)'], c= labels.astype(np.float), s=200)

#note that it will open in a separate window. Drag the graph and rotate with your mouse to see the results interactively.

#better to download it to your own computer to visualise it properly

#%matplotlib inline    #to add after finishing to go back to charts inside the notebook.

Let's just make sure that 5 is the optimal number of clusters by constructing an elbow plot

In [ ]:
%matplotlib inline   

elbowlist3 = []
for i in range(1,15): 
    k_means_3D = KMeans(n_clusters=i, init="k-means++",random_state=0)
    k_means_3D.fit(cust_income)
    elbowlist3.append(k_means_3D.inertia_)  

plt.plot(range(1,15),elbowlist3,marker="*",c="black")
plt.title("Elbow plot for optimal number of clusters: age, income and spending")

Here as well we can notice that the cluster number 5 that we chose is correct. The elbow plot has a distinct slope break at 5 indicating that 5 is the optimal cluster number when comparing between age, income and spending.

In [ ]:
score_3 = silhouette_score (cust_3D, k_means_3D.labels_)

print("The score is = ", score_3)

<p>The 3D graph is highly informative because we can visualise 3 dimensions in a single graph. Usually, using multiple dimensions in clustering is very challeging, but here we succeeded in visualising 3 dimensions by using a 3D graph. In this case, we linked the age, the income and the spending and we got better results. The 5 customer segments that use the mall are as follow:    
    <ul>
        <li>Young (20-40 yo) , high income and high spending (yellow)</li>
        <li>Young (15 to 30 yo), low income and high spending (purple)</li>
        <li>High income and low spending, they don't have a specific age group (light blue)</li>
        <li>Low income and low spending, they don't have a specific age group (green)</li>
        <li>Average income, average spending, they don't have a specific age group (dark blue)</li>
    </ul>
        
*note that young is considered here as being less than 40

In this cluster model, we have a silhouete score of 0.44. This is less than the previous scores we had. This might be, however, due to the fact that we are measuring in a 3D environment, where the distance between the data points and the origin of the coordinate system grows as a square root of the number of dimensions D. Consequently, the mean distance between data points diverges and looses its meaning which in turn leads to the divergence of the Euclidean distance, the most common distance used for clustering. Therefore, it is better to neglect the fact that this score is the least in our examples. 

--> --> --> Check that nice article by Nikolay Oskolkov on high dimension clustering: https://towardsdatascience.com/how-to-cluster-in-high-dimensions-4ef693bacc6

        

## Summary

<p>K-mean clustering has been performed over a mall customer dataset to classify customers into different segments. Five customer segments were found having different age, income and spending trends. In order to make it better for the mall management to retain customers and increase sales, it is recommended that management focuses on retaining the following segments:
<ul>
    <li>Rich and high spending people between their 20 and 40's</li>
    <li>Relatively poor and high spending people between their 15 and 30's</li>
    </ul>

## If you like this notebook, please upvote